In [1]:
# load and the test weekly fpl scrape
season="2019-20"
gw=10
model_type="rf"

In [2]:
import os
import pandas as pd
import numpy as np
from keras.models import load_model
import pickle
from sklearn.externals import joblib

Using TensorFlow backend.


In [3]:
os.chdir(os.path.join(os.getcwd(), "..", "..", "data"));

In [4]:
# scaler = joblib.load("scaler.save")

In [5]:
player_id_df = pd.read_csv("player_id_map.csv");

In [6]:
team_id_df = pd.read_csv("team_id_map.csv");

In [7]:
X_test = pd.read_csv("x-test.csv");

In [8]:
os.chdir(os.path.join(os.getcwd(), ".."))

In [9]:
model_dir = os.path.join(os.getcwd(), "model", season, str(gw))

In [10]:
lineup_dir = os.path.join(os.getcwd(), "predicted-lineup", season, str(gw))

In [11]:
os.chdir(model_dir)

In [12]:
if model_type == "nn":
    model = load_model("{}-model.h5".format(model_type))
else:
    with open("{}-model.pkl".format(model_type), 'rb') as f:
            model = pickle.load(f)

In [13]:
os.chdir(lineup_dir)

In [14]:
weekly_scrape = pd.read_csv("gw{}-fpl-data.csv".format(gw));

In [15]:
weekly_scrape_scaled = weekly_scrape[X_test.columns];
# scaler.transform(weekly_scrape[X_test.columns]);

In [16]:
y_pred_weekly = model.predict(weekly_scrape_scaled);

In [17]:
y_pred_weekly_df = pd.DataFrame({'player_id': list(weekly_scrape["player_id"]), 
                                 'position': list(weekly_scrape["position"]),
                                 'predicted': y_pred_weekly.flatten()});

In [18]:
player_names = [];
opponent_names = [];

for index, row in weekly_scrape.iterrows():
    player_id = row["player_id"];
    opponent_id = row["opponent_id"];
    
    player_names.append(player_id_df[player_id_df["player_id"] == player_id]["actual_name"].iloc[0]);
    opponent_names.append(team_id_df[team_id_df["team_id"] == opponent_id]["team_name"].iloc[0]);

In [19]:
y_pred_weekly_df["player_name"] = player_names;
y_pred_weekly_df["opponent"] = opponent_names;

In [20]:
y_pred_weekly_df = y_pred_weekly_df[["player_id", "position", "player_name", "opponent", "predicted"]];

In [21]:
y_pred_weekly_df.sort_values(["predicted"], ascending=False)

,player_id,position,player_name,opponent,predicted
7,585,MID,Sadio Mané,Tottenham Hotspur,3.367586
10,385,MID,Kevin De Bruyne,Aston Villa,3.352451
17,304,FWD,Jamie Vardy,Southampton,3.324969
16,161,FWD,Danny Ings,Leicester City,3.274737
2,648,DEF,Trent Alexander-Arnold,Tottenham Hotspur,3.205010
4,977,DEF,Issa Diop,Sheffield United,3.105472
14,858,FWD,Pierre-Emerick Aubameyang,Crystal Palace,3.031038
12,882,FWD,Tammy Abraham,Burnley,2.956601
13,1211,FWD,Teemu Pukki,Manchester United,2.923175
18,17,MID,Adama Traoré,Newcastle United,2.872443


In [22]:
y_pred_weekly_df.to_csv("gw{}-predicted-data-{}.csv".format(gw, model_type));

In [23]:
min_max_pos = {
    "GK": {
        "min": 1,
        "max": 1
    },
    "DEF": {
        "min": 3,
        "max": 5
    },
    "MID": {
        "min": 2,
        "max": 5
    },
    "FWD": {
        "min": 1,
        "max": 3
    }
}

In [24]:
def best11(df, max_players=11):
    selectedTeam = pd.DataFrame(columns=df.columns);
    positions = list(np.unique(df["position"]));
    
    # base cases - min 1 keeper, 3 def, 2 mids, 1 fwd
    for position in positions:
        df_ = df[df["position"] == position].sort_values(["predicted"], 
                                                         ascending=False).head(min_max_pos[position]["min"]);
        selectedTeam = selectedTeam.append(df_);
        
    gk_ind = positions.index("GK");
    del positions[gk_ind];
    
    num_players = selectedTeam.shape[0];
    
    # recursive case - max 5 def, 5 mids, 3 fwd
    while num_players < max_players:
        num_def = selectedTeam[selectedTeam["position"] == "DEF"].shape[0];
        num_mid = selectedTeam[selectedTeam["position"] == "MID"].shape[0];
        num_fwd = selectedTeam[selectedTeam["position"] == "FWD"].shape[0];
        
        candidates=[]

        for position in positions:
            num_position = selectedTeam[selectedTeam["position"] == position].shape[0];
            
            if (num_position < min_max_pos[position]["max"]):
                df_ = df[df["position"] == position].sort_values(["predicted"], ascending=False).head(selectedTeam[selectedTeam["position"] == position].shape[0] + 1).tail(1)
                candidates.append(df_)
    
        max_score = candidates[0]["predicted"].iloc[0]
        best_player = candidates[0]
        
        for index in range(len(candidates)):
            score = candidates[index]["predicted"].iloc[0]
            if (score > max_score):
                max_score = score
                best_player = candidates[index]
        
        selectedTeam = selectedTeam.append(best_player);
        
        num_players = num_players + 1;
        
    
    best11 = pd.DataFrame(columns=selectedTeam.columns);
    best11 = best11.append(selectedTeam[selectedTeam["position"] == "GK"]);
    best11 = best11.append(selectedTeam[selectedTeam["position"] == "DEF"]);
    best11 = best11.append(selectedTeam[selectedTeam["position"] == "MID"]);
    best11 = best11.append(selectedTeam[selectedTeam["position"] == "FWD"]);
    
    top2_ = list(best11.sort_values(["predicted"], ascending=False).head(2).index);
    best11["roles"] = ""
    best11.at[top2_[0], 'roles']="C"
    best11.at[top2_[1], 'roles']="VC"
    
    best11["predicted"] = best11["predicted"].apply(lambda predicted: round(predicted, 0));
    
    best11["predicted"] = best11.apply(lambda row: row["predicted"] * 2 if row["roles"] == "C" else row["predicted"], axis=1)
    
    return best11


In [25]:
best11 = best11(y_pred_weekly_df)

In [26]:
best11

,player_id,position,player_name,opponent,predicted,roles
0,832,GK,Mathew Ryan,Everton,3.0,
2,648,DEF,Trent Alexander-Arnold,Tottenham Hotspur,3.0,
4,977,DEF,Issa Diop,Sheffield United,3.0,
15,439,DEF,Marcos Alonso,Burnley,3.0,
7,585,MID,Sadio Mané,Tottenham Hotspur,6.0,C
10,385,MID,Kevin De Bruyne,Aston Villa,3.0,VC
18,17,MID,Adama Traoré,Newcastle United,3.0,
11,1213,MID,Todd Cantwell,Manchester United,3.0,
17,304,FWD,Jamie Vardy,Southampton,3.0,
16,161,FWD,Danny Ings,Leicester City,3.0,


In [27]:
sum(best11["predicted"])

36.0

In [28]:
best11.to_csv("gw{}-best11-{}.csv".format(gw,model_type), index=False);